In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 52.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylin

In [2]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

In [3]:
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

In [4]:
model_name = "meta-llama/Meta-Llama-3-8B"

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [8]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [9]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [10]:
prompt = "What is Machine Learning?"

In [11]:
llama3_response = get_response(prompt)
llama3_response

'What is Machine Learning? How can you use it in your business? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it'

In [12]:
print(llama3_response)

What is Machine Learning? How can you use it in your business? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it


In [13]:
print(llama3_response[len(prompt):])

 How can you use it in your business? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it in your business? What is Machine Learning? What are the different types of Machine Learning? What are the different types of Machine Learning? How can you use it


In [18]:
def identify_emotion(prompt):
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate a response from the model
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [19]:
prompt = "Analyze the following text and identify the primary emotion: 'I am feeling very happy and excited today!'"

In [20]:
emotion = identify_emotion(prompt)
print(f"Identified Emotion: {emotion}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Identified Emotion: Analyze the following text and identify the primary emotion: 'I am feeling very happy and excited today!' The text was taken from a person who has been diagnosed with bipolar disorder. If you are feeling very happy and excited today, you are likely to have a high mood. If you are feeling very happy and excited today, you are likely to have a low mood. If you are feeling very happy and excited today, you are likely to have a mixed mood. If you are feeling very happy


In [21]:
def identify_filler_words(prompt):
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate a response from the model
    outputs = model.generate(inputs["input_ids"], max_length=100, num_return_sequences=1)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [22]:
text_to_analyze = "So, um, I think we should, like, go to the park, you know?"
prompt = f"Identify the filler words in the following text: '{text_to_analyze}'"

In [23]:
filler_words = identify_filler_words(prompt)
print(f"Identified Filler Words: {filler_words}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Identified Filler Words: Identify the filler words in the following text: 'So, um, I think we should, like, go to the park, you know?'


In [24]:
def generate_emotion_prompt(text):
    prompt = f"Identify the emotion in the following text:\n\nText: {text}\n\nEmotion:"
    return prompt

In [25]:
def predict_emotion(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_emotion_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=50)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the emotion from the response
    emotion = response.split("Emotion:")[-1].strip()
    return emotion

In [26]:
text_to_classify = "I feel so happy today!"
predicted_emotion = predict_emotion(text_to_classify)
print("Predicted Emotion:", predicted_emotion)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Predicted Emotion: Happy

Explanation: The text mentions the emotion "happy". The emotion is expressed in the first person, and is expressed positively.


Filler words

In [27]:
def generate_filler_prompt(text):
    prompt = f"Identify the filler words in the following text:\n\nText: {text}\n\nFiller words:"
    return prompt

In [28]:
def predict_filler_words(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_filler_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the filler words from the response
    filler_words = response.split("Filler words:")[-1].strip()
    return filler_words

In [29]:
text_to_classify = "I mean, like, you know, it's sort of interesting, right?"
predicted_filler_words = predict_filler_words(text_to_classify)
print("Filler Words:", predicted_filler_words)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Filler Words: mean, like, you know, it's, sort of, right?


Grammers

In [31]:
def generate_grammar_prompt(text):
    prompt = f"Identify the grammar mistakes in the following text:\n\nText: {text}\n\nGrammar mistakes:"
    return prompt

In [32]:
def predict_grammar_mistakes(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_grammar_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the grammar mistakes from the response
    grammar_mistakes = response.split("Grammar mistakes:")[-1].strip()
    return grammar_mistakes

In [33]:
text_to_classify = "This is an example sentence with a grammatical error are."
predicted_grammar_mistakes = predict_grammar_mistakes(text_to_classify)
print("Grammar Mistakes:", predicted_grammar_mistakes)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Grammar Mistakes: This is an example sentence with a grammatical error are.

Explanation: The correct sentence is: This is an example sentence with a grammatical error are.

The correct sentence is: This is an example sentence with a grammatical error are.


Formal and Informal

In [34]:
def generate_formality_prompt(text):
    prompt = f"Classify the following text as formal or informal:\n\nText: {text}\n\nClassification:"
    return prompt

In [35]:
def predict_formality(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    prompt = generate_formality_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=50)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the classification from the response
    classification = response.split("Classification:")[-1].strip()
    return classification

In [36]:
text_to_classify = "Hey buddy, what's up? Wanna hang out later?"
predicted_formality = predict_formality(text_to_classify)
print("Classification:", predicted_formality)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Classification: Informal

Text:
